In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

1. set의 특징 : 중복 허용 x, 순서가 없다 -> 인덱싱으로 값을 얻을 수 없다(딕셔너리와 유사): 리스트나 튜플로 변환 후 인덱싱으로 접근

https://blogofth-lee.tistory.com/269 참고

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터
sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [3]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {c: i for i,c in enumerate(world_set)}

In [4]:
print(world_set)

['.', 'a', 'm', 'y', ',', 'f', 's', 't', ' ', 'k', 'l', 'n', 'r', 'i', 'h', 'b', 'p', 'u', 'c', 'o', "'", 'g', 'e', 'w', 'd', 'B']


In [5]:
print(vocab)

{'.': 0, 'a': 1, 'm': 2, 'y': 3, ',': 4, 'f': 5, 's': 6, 't': 7, ' ': 8, 'k': 9, 'l': 10, 'n': 11, 'r': 12, 'i': 13, 'h': 14, 'b': 15, 'p': 16, 'u': 17, 'c': 18, 'o': 19, "'": 20, 'g': 21, 'e': 22, 'w': 23, 'd': 24, 'B': 25}


In [6]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [7]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [8]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce
## 아래 결과 코드를 보면 x와 y의 관계가 한칸 차이임을 알 수 있다.
for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [9]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[25, 12, 13, 18, 9, 8, 23, 1, 10, 10]
[12, 13, 18, 9, 8, 23, 1, 10, 10, 6]


In [10]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

<ipython-input-10-a1581d86cfe7>:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  X = torch.FloatTensor(x_one_hot)


In [11]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [12]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [13]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([12, 13, 18,  9,  8, 23,  1, 10, 10,  6])


In [14]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_first 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim,hidden_size=hidden_dim,batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim,hidden_dim,bias=True)
  
  def forward(self, x):
    x,_status = self.rnn(x)
    x=self.fc(x)
    return x

In [15]:
net = Net(vocab_size, hidden_size, 2)

In [16]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [17]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [18]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

e'iiiiiieiiiiiiiii'iiiiiiiiiiiieiiiii'ii'iiiiiiii'iiiiiiiiiiieiii'iiiiiiiiieiiiiiiiiiiii'iiiiiii'iiiiiiiiiiiiiiiiiei'iiiiiiiiiiiiiiiiieiii'iiiiiiiiieiiiiiiiii'iiiiiiiieiiiiiieiiiieiiiiiiiiiiiiiii'e
e'eeeereeeeeeereeeeireeeieeeieeeeeeeeee eeieeeeeeeeeeeeeeeeeeeeeeeerieeeereeeeeeereeeeeeeireeeeeeeeeeeieeeieeeeeieeeeieeeeeeeeireeeeeeeeeeeerieeeereeeeeeereeeeireeeeeeeeeeeeeerereee eueereeeeeeeeee
eeeeeereeeeetereeerereeeeeteereeeeeteee eeeeeeeeeeeeeeeeeeeeeeeeerereeeeereeeeetereeeeeerereeeeeereeeeeeee ereeeeeeereeeeeeeeeere eeeeeeeerereeeeereeeeetereeerereeeeeeeeeeeeeerertee ereereeeeeeeeee
eeeeeereeeeetereterereteeeteerteeeeteee t e eeeeeeeeteeeeeeeteeterere eeereeeeetereeeeterereteteereeteeeee erte eteereeeeeteeeere eeeteeterere eeereeeeetereterereteteee eteeterertte treereeeeteeeee
ee eeertt  eterttererete tttertt  ttett t t e tttettte  eetete terere teertt  eterettttereretetter ttett t t tt etetrett tte  ert ttete terere teertt  eterttereretettet ttette trtt  tt ttt ttte   e
ee t tttt 

In [19]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinks,"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

https://sputnik-kr.tistory.com/239 참조

In [47]:
#형태소 분석을 위한 konlpy를 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
sentence = ("오늘 아침 새로운 원두가 회사에 들어왔다."
            "새 원두로 내린 커피는 향이 진했고 맛있어 보였다. 그러나"
            "나는 커피를 잘 못 마시는 사람이라서 한 입만 먹었다.")
    

In [55]:
#한국어 형태소 분석기로 문장 토큰화 
from konlpy.tag import *
okt = Okt()  
token = okt.morphs(''.join(sentence))  
print(token)

['오늘', '아침', '새로운', '원두', '가', '회사', '에', '들어왔다', '.', '새', '원두', '로', '내린', '커피', '는', '향', '이', '진했고', '맛있어', '보였다', '.', '그러나나는', '커피', '를', '잘', '못', '마시는', '사람', '이라서', '한', '입', '만', '먹었다', '.']


In [66]:
vocab = {c: i for i,c in enumerate(token)}
print(vocab)

{'오늘': 0, '아침': 1, '새로운': 2, '원두': 10, '가': 4, '회사': 5, '에': 6, '들어왔다': 7, '.': 33, '새': 9, '로': 11, '내린': 12, '커피': 22, '는': 14, '향': 15, '이': 16, '진했고': 17, '맛있어': 18, '보였다': 19, '그러나나는': 21, '를': 23, '잘': 24, '못': 25, '마시는': 26, '사람': 27, '이라서': 28, '한': 29, '입': 30, '만': 31, '먹었다': 32}


In [58]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 30


In [59]:
def one_hot_encoding(word, vocab):
  one_hot_vector = [0]*(len(vocab))
  index = vocab[word]
  one_hot_vector[index] = 1
  return one_hot_vector

In [64]:
one_hot_encoding("커피", vocab)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0]

한글 인코딩 실패함.-> 영어로 진행..!

In [70]:
#1. 생성할 문장 데이터
sentence = ("You can do anything if you want to"
            "do and never give up even though you meet a difficulty, and if so"
            "you will eventually reach to your final goal.")

In [71]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {c: i for i,c in enumerate(world_set)}

In [72]:
print(world_set)

['.', 'a', 'm', 'y', ',', 'f', 's', 't', ' ', 'v', 'l', 'n', 'r', 'i', 'Y', 'h', 'p', 'u', 'c', 'o', 'g', 'e', 'w', 'd']


In [73]:
print(vocab)

{'.': 0, 'a': 1, 'm': 2, 'y': 3, ',': 4, 'f': 5, 's': 6, 't': 7, ' ': 8, 'v': 9, 'l': 10, 'n': 11, 'r': 12, 'i': 13, 'Y': 14, 'h': 15, 'p': 16, 'u': 17, 'c': 18, 'o': 19, 'g': 20, 'e': 21, 'w': 22, 'd': 23}


In [74]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 24


In [76]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 15  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [77]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce
## 아래 결과 코드를 보면 x와 y의 관계가 한칸 차이임을 알 수 있다.
for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 You can do anyt -> ou can do anyth
1 ou can do anyth -> u can do anythi
2 u can do anythi ->  can do anythin
3  can do anythin -> can do anything
4 can do anything -> an do anything 
5 an do anything  -> n do anything i
6 n do anything i ->  do anything if
7  do anything if -> do anything if 
8 do anything if  -> o anything if y
9 o anything if y ->  anything if yo
10  anything if yo -> anything if you
11 anything if you -> nything if you 
12 nything if you  -> ything if you w
13 ything if you w -> thing if you wa
14 thing if you wa -> hing if you wan
15 hing if you wan -> ing if you want
16 ing if you want -> ng if you want 
17 ng if you want  -> g if you want t
18 g if you want t ->  if you want to
19  if you want to -> if you want tod
20 if you want tod -> f you want todo
21 f you want todo ->  you want todo 
22  you want todo  -> you want todo a
23 you want todo a -> ou want todo an
24 ou want todo an -> u want todo and
25 u want todo and ->  want todo and 
26  want todo and  -> 

In [78]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[14, 19, 17, 8, 18, 1, 11, 8, 23, 19, 8, 1, 11, 3, 7]
[19, 17, 8, 18, 1, 11, 8, 23, 19, 8, 1, 11, 3, 7, 15]


In [79]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [80]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([129, 15, 24])
레이블의 크기 : torch.Size([129, 15])


In [81]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [82]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([19, 17,  8, 18,  1, 11,  8, 23, 19,  8,  1, 11,  3,  7, 15])


In [83]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_first 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim,hidden_size=hidden_dim,batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim,hidden_dim,bias=True)
  
  def forward(self, x):
    x,_status = self.rnn(x)
    x=self.fc(x)
    return x

In [84]:
net = Net(vocab_size, hidden_size, 2)

In [85]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [86]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([129, 15, 24])


In [87]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

ftetetetetttetfttettt tttetttetteteettettetetetttfttetrtttetettetetttetertttttet ttettettttette tetteetttrrtttteeetrrttetteetettteett tetrttttr
t ete e e  tetette  t tt ettte  e e  tet e  te  t ttet   te et  te t ete tett et  t tt ttttet e te teette   t te et  tte t e e tt et   et    e 
t  t                   t  tt                         t                te                              tt          t    e           t           
                                                                                                                                               
                                                                                                                                               
                                                                                                                                               
                                                                                                                                        

In [88]:
predict_str

'ou wantdo anything if you want todo and never give up even though you meet a difficulty, and if soyou will eventually reach to your final giall'